# Final Long Short-Term Memory (LSTM) Model Deploy

### Import Necessary Modules

In [1]:
import librosa
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import OneHotEncoder

## Final Prediction Script

In [4]:
# Load the trained emotion recognition model
model = load_model('final_LSTM_best_model.h5')

# Load the encoder for emotion labels
## load the same encoder that was used during model training
## Define the emotion labels here (in the same order as used during training)
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'ps']
enc = OneHotEncoder(sparse_output=False)
enc.fit(np.array(emotion_labels).reshape(-1, 1))

# Create a mapping from emotion labels to integers
label_to_int = {label: i for i, label in enumerate(emotion_labels)}

# Function to extract MFCC features from an audio segment
def extract_mfcc(y, sr):
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

# Function to predict emotion for a single segment
def predict_emotion(segment):
    # Extract MFCC features
    mfcc = extract_mfcc(segment, sampling_rate)
    
    # Reshape for model input
    mfcc = mfcc.reshape(1, mfcc.shape[0], 1)
    
    # Predict emotion
    emotion_probabilities = model.predict(mfcc)
    
    return emotion_probabilities


# Load a audio file (replace with the audio file)
audio_path = './TestingAudio/audio1.wav'

# Split long audio into overlapping segments
segment_length = 3  # Length of each segment in seconds
overlap = 0.0  # Overlap between segments in seconds
sampling_rate = 22050  # Replace with the appropriate sampling rate
segments = []

# Split the audio into overlapping segments
y, _ = librosa.load(audio_path, sr=sampling_rate)
for start in np.arange(0, len(y), int((segment_length - overlap) * sampling_rate)):
    end = int(start + segment_length * sampling_rate)
    segment = y[start:end]
    segments.append(segment)

# Predict emotions for all segments
emotion_predictions = []
for segment in segments:
    emotion_probabilities = predict_emotion(segment)
    emotion_predictions.append(emotion_probabilities)

# Convert aggregated probabilities to emotion labels for all segments
emotion_labels_for_segments = [enc.inverse_transform(emotion_probabilities).flatten()[0] for emotion_probabilities in emotion_predictions]

# Calculate the percentage of each emotion
total_segments = len(emotion_labels_for_segments)
emotion_percentages = {label: emotion_labels_for_segments.count(label) / total_segments * 100 for label in set(emotion_labels_for_segments)}

# Return the emotions and their percentages
emotions_and_percentages = {emotion: percentage for emotion, percentage in emotion_percentages.items()}

for emotion, percentage in emotions_and_percentages.items():
    print(f'{emotion}: {percentage:.2f}%')

1/1 [==============================] - 0s 22ms/step
sad: 12.50%
fear: 12.50%
disgust: 75.00%
